In [ ]:
%%capture
!pip install langchain==0.1.1 openai==1.8.0 langchainhub langchain-openai

In [ ]:
import os
import getpass

os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass("Enter Hugging Face API Token:")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from langchain_community.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def run_llm_chain(repo_id: str,
                  prompt:PromptTemplate,
                  input: str,
                  temperature: float = 1e-3,
                  max_length: int = 4096,
                  do_sample: bool = True) -> str:
    """
    Initializes a HuggingFaceHub with specified parameters and runs an LLMChain with a given question.

    Args:
    repo_id (str): The repository ID for the HuggingFace model.
    temperature (float, optional): The temperature for text generation. Default is 0.5.
    max_length (int, optional): The maximum length of the generated text. Default is 64.

    Returns:
    str: The result from running the LLMChain.
    """
    # Define the prompt template within the function
    prompt = prompt
    llm = HuggingFaceHub(
        repo_id=repo_id,
        model_kwargs={
            "temperature": temperature,
            "max_length": max_length,
            "do_sample":do_sample,
            "num_beams":5,
            "early_stopping":True
            }
    )
    llm_chain = prompt | llm | StrOutputParser()
    return llm_chain.invoke({"input":input})

In [ ]:
instruct_model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# 🎯 **Clarity is Essential**

- Avoid ambiguity for precise model responses.

- Use distinct delimiters (triple quotes, backticks, angle brackets) to separate instructions clearly.

- This prevents confusion between directives and general content.

In [ ]:
text_to_summarize = f"""It's important to give clear and specific instructions \
when asking a model to perform a task. This will help guide the model towards \
the desired output, and minimize the chances of receiving irrelevant or \
incorrect responses. It's important to note that writing a clear prompt  \
doesn't necessarily mean writing a short one. In fact, longer prompts can  \
often provide more clarity and context for the model, resulting in more \
detailed and relevant outputs.
"""

template = """Summarize the text delimited by triple backticks into a single sentence: ```{input}```.
Summary:
"""

In [ ]:
prompt = PromptTemplate.from_examples(
    examples=[template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )

In [8]:
prompt.pretty_print()

<s>[INST]

Summarize the text delimited by triple backticks into a single sentence: ```{input}```. 
Summary:


[/INST]


In [9]:
result = run_llm_chain(repo_id = instruct_model_id,
              prompt = prompt,
              input = text_to_summarize)

In [10]:
print(result)

<s>[INST]

Summarize the text delimited by triple backticks into a single sentence: ```It's important to give clear and specific instructions when asking a model to perform a task. This will help guide the model towards the desired output, and minimize the chances of receiving irrelevant or incorrect responses. It's important to note that writing a clear prompt  doesn't necessarily mean writing a short one. In fact, longer prompts can  often provide more clarity and context for the model, resulting in more detailed and relevant outputs.
```. 
Summary:


[/INST] Providing clear and detailed instructions when asking a model to perform a task is crucial for accurate and relevant responses, and longer prompts can offer more context and clarity.


# 🔍 **Request Structured Output for Easy Interpretation**

- Prioritize clarity: Ask for outputs in structured formats (like JSON, XML).

- Structured data is simpler to interpret and use.

- Defining the desired format upfront leads to more organized
results.

In [11]:
text = """Create some imaginary movie titles, complete with directors and genres.
Format the information in JSON, using the keys: 'movie_id', 'title', 'director', and 'genre'.
"""


In [12]:
prompt = PromptTemplate.from_examples(
    examples=[text],
    input_variables=[],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )

In [13]:
prompt.pretty_print()

<s>[INST]

Create some imaginary movie titles, complete with directors and genres.
Format the information in JSON, using the keys: 'movie_id', 'title', 'director', and 'genre'.


[/INST]


In [14]:
result = run_llm_chain(repo_id = instruct_model_id,
              prompt = prompt,
              input = text_to_summarize)

In [15]:
print(result)

<s>[INST]

Create some imaginary movie titles, complete with directors and genres.
Format the information in JSON, using the keys: 'movie_id', 'title', 'director', and 'genre'.


[/INST] [
  {
    "movie_id": 1,
    "title": "The Time Traveler's Paradox",
    "director": "Christopher Nolan",
    "genre": "Sci-Fi Thriller"
  },
  {
    "movie_id": 2,
    "title": "Love in the Time of Zombies",
    "director": "Zack Sny


# 🔍 **Process for Verifying Conditions**

1. **Spot the Criteria**: Look for specific requirements, like a set of instructions.

2. **Assess Compliance**: Examine the text to see if it meets these criteria.

3. **Customized Reply**: React accordingly - proceed if criteria are met, or take a different route if not.

4. **Transparent Feedback**: Clearly explain if conditions were met and the logic behind your response.

In [16]:
text ="""To reset your password, first, navigate to the settings page. Click \
on the 'Security' tab, and then select 'Change Password.' Enter your current \
password, followed by your new password twice for confirmation. Click on the \
'Save Changes' button to update your password.
"""

template = """You will be provided with text enclosed in angle brackets (<>). Determine \
if the text contains a procedural guide.  If it does, convert the guide into a \
numbered list format with a title:

<Title>

1. ...
2. ...
3. ...
4. ...

If the text does not contain a procedural guide, respond with 'No procedural guide found.'

<{input}>
"""


In [17]:
prompt = PromptTemplate.from_examples(
    examples=[template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix="[/INST]"
    )

In [18]:
prompt.pretty_print()

<s>[INST]

You will be provided with text enclosed in angle brackets (<>). Determine if the text contains a procedural guide.  If it does, convert the guide into a numbered list format with a title:

<Title>

1. ...
2. ...
3. ...
4. ...

If the text does not contain a procedural guide, respond with 'No procedural guide found.'

<{input}>


[/INST]


In [19]:
result = run_llm_chain(repo_id = instruct_model_id,
              prompt = prompt,
              input = text)

In [20]:
print(result)

<s>[INST]

You will be provided with text enclosed in angle brackets (<>). Determine if the text contains a procedural guide.  If it does, convert the guide into a numbered list format with a title:

<Title>

1. ...
2. ...
3. ...
4. ...

If the text does not contain a procedural guide, respond with 'No procedural guide found.'

<To reset your password, first, navigate to the settings page. Click on the 'Security' tab, and then select 'Change Password.' Enter your current password, followed by your new password twice for confirmation. Click on the 'Save Changes' button to update your password.
>


[/INST] Title: Resetting Your Password

1. Navigate to the settings page.
2. Click on the 'Security' tab.
3. Select 'Change Password'.
4. Enter your current password.
5. Enter your new password twice for confirmation.
6. Click on the 'Save Changes' button.


# 🤔 **Strategies for Complex Queries**

1. **Detail Steps & Format**: Outline each step to complete a task and specify the output format.

2. **Break Down Tasks**: Like humans, the model can benefit from step-by-step instructions for complex tasks.

3. **Logical Sequencing**: Request a logical flow or reasoning chain before the final answer.

4. **Think Before Concluding**: Instruct the model to fully process the task before jumping to conclusions.

In [22]:
template = """Your task is to determine if a student's solution is correct or incorrect.\

To solve the problem do the following:

- First, work out your own solution to the problem including the final total.

- Then compare your solution to the student's solution and evaluate if the student's\
solution is correct or not.

Don't decide if the student's solution is correct until  you have done the problem yourself.

Question: ```{input}```

"""

suffix = """Respond with the following format:

Question: ```question here```

Student's solution: ```student's solution here```

Actual solution: ```steps to work out the solution and your solution here```

Is the student's solution the same as actual solution just calculated:```yes or no```

Student grade: ```correct or incorrect```

Think step-by-step, and assess provide your solution to the question.

My solution is: [/INST]
"""


question = """```I'm building a solar power installation and I need help working out the financials:
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year,\
and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.```

Student's solution:```Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x

Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000```
"""


In [23]:
prompt = PromptTemplate.from_examples(
    examples=[template],
    input_variables=["input"],
    prefix = "<s>[INST]",
    suffix=suffix
    )

In [24]:
prompt.pretty_print()

<s>[INST]

Your task is to determine if a student's solution is correct or incorrect.
To solve the problem do the following:

- First, work out your own solution to the problem including the final total.

- Then compare your solution to the student's solution and evaluate if the student'ssolution is correct or not.

Don't decide if the student's solution is correct until  you have done the problem yourself.

Question: ```{input}```



Respond with the following format:

Question: ```question here```

Student's solution: ```student's solution here```

Actual solution: ```steps to work out the solution and your solution here```

Is the student's solution the same as actual solution just calculated:```yes or no```

Student grade: ```correct or incorrect```

Think step-by-step, and assess provide your solution to the question.

My solution is: [/INST]



In [25]:
result = run_llm_chain(repo_id = instruct_model_id,
              prompt = prompt,
              input = question)

In [26]:
print(result)

<s>[INST]

Your task is to determine if a student's solution is correct or incorrect.
To solve the problem do the following:

- First, work out your own solution to the problem including the final total.

- Then compare your solution to the student's solution and evaluate if the student'ssolution is correct or not.

Don't decide if the student's solution is correct until  you have done the problem yourself.

Question: ``````I'm building a solar power installation and I need help working out the financials:
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year,and an additional $10 / square foot

What is the total cost for the first year of operations as a function of the number of square feet.```

Student's solution:```Let x be the size of the installation in square feet.

Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x

Total cost: 100x + 

# 🎯 **Balance Context and Clarity**

- Provide essential information, but avoid overloading with details.

- Focus on key points in your prompt.

- The model can ask for more context if needed.

# Summary

1. 🎯 **Clarity is Essential**: Use clear delimiters to avoid ambiguity and ensure precise responses from the model.

2. 🔍 **Request Structured Output**: Ask for data in organized formats like JSON or XML for ease of interpretation and use.

3. 🔍 **Process for Verifying Conditions**:
   - **Spot the Criteria**: Identify specific requirements in the text.
   - **Assess Compliance**: Check if the text meets these criteria.
   - **Customized Reply**: Respond based on the fulfillment of conditions.
   - **Transparent Feedback**: Clearly communicate the outcome and reasoning.

4. 🤔 **Strategies for Complex Queries**:
   - **Detail Steps & Format**: Define steps and output format for complex tasks.
   - **Break Down Tasks**: Use step-by-step instructions for better processing.
   - **Logical Sequencing**: Ask for a logical flow before the final answer.
   - **Think Before Concluding**: Encourage thorough processing before conclusions.

5. 🎯 **Balance Context and Clarity**: Provide essential information without overloading, allowing the model to request more context if needed.